In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import sys
from html.parser import HTMLParser
from html.entities import name2codepoint
sns.set(color_codes=True)
import warnings
warnings.filterwarnings("ignore")                   
import nltk                                         
nltk.download('stopwords')
from nltk.corpus import stopwords                   
from nltk.stem import PorterStemmer  
LA = np.linalg

from sklearn.feature_extraction.text import CountVectorizer          
from sklearn.feature_extraction.text import TfidfVectorizer          
from gensim.models.word2vec import Word2Vec                                  
import re
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/chd415/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load training data text
def load_data(filename):
    load_file = pd.read_csv(filename,delimiter=',', header=0,
                        dtype={'name':str, 'lvl1':str, 'lvl2':str, 'lvl3':str, 'descrption':str, 'type':str})
    load_file.columns = ['id', 'name','lvl1','lvl2','lvl3','descrption','price','type']
    load_file.duplicated(subset=None, keep='first')
    load_file.set_index('id', inplace = True)
    load_file.head()
    return load_file
#print(len(train_file))
def load_label(filename):
    load_label = pd.read_csv(filename,delimiter=',', header=0)
    load_label.columns = ['id', 'score']
    load_label.duplicated(subset=None, keep='first')
    load_label.set_index('id', inplace = True)
    return load_label

In [3]:
def map_mathod(column):
    values = []
    indexs = []
    mapping = {}
    index = 0
    for count in range(len(train_file)):
        value = train_file.get_value(count+1,column)
        if value in values and value != np.nan:
            continue
        values.append(value)
        indexs.append(len(values))
    for j in range(len(indexs)):
        mapping[values[j]] = indexs[j]
    mapping[np.nan] = 0.0
    return mapping
#train_file['lvl3'] = train_file['lvl3'].str.lower().replace('[^\'\w]+',' ',regex=True)
#mapping_lvl3 = map_mathod('lvl3')
#print(mapping_lvl3)

#from sktlearn import preprocessing
#le = preprocessing.LabelEncoder()
#fit_transform

In [4]:
def text_embedding(column):
    temp_X = column.astype(str)
    stop = set(stopwords.words('english'))
    temp =[]
    snow = nltk.stem.SnowballStemmer('english')

    for sentence in temp_X: 
        words = [snow.stem(word) for word in sentence.split(' ') if word not in stopwords.words('english')]   # Stemming and removing stopwords
        temp.append(sentence)

    count_vect = CountVectorizer(max_features=10000)
    bow_data = count_vect.fit_transform(temp)

    final_tf = temp
    tf_idf = TfidfVectorizer(max_features=10000)
    tf_data = tf_idf.fit_transform(final_tf)
    w2v_data = temp
    splitted = []
    for row in w2v_data: 
        splitted.append([word for word in row.split()])     #splitting words
    
    train_w2v = Word2Vec(splitted,min_count=1,size=50, workers=4)
    avg_data = []
    for row in splitted:
        vec = np.zeros(50, dtype=float)
        count = 0
        for word in row:
            try:
                vec += train_w2v[word]
                count += 1
            except:
                pass
        if (count == 0):
            avg_data.append(vec)
        else:
            avg_data.append(vec/count)
#        avg_data.append(vec)
    
    return avg_data

In [ ]:
#test cell
temp = load_data('train_data.csv')
temp['descrption'] = temp['descrption'].str.lower()
description_X = temp.descrption.str.lower().replace('<.*?>','',regex=True).replace('[^\w\s]+',' ',regex=True)


In [ ]:
#test cell
def test_embedding(column):
    temp_X = column.astype(str)
    stop = set(stopwords.words('english'))
    temp =[]
    snow = nltk.stem.SnowballStemmer('english')

    for sentence in temp_X: 
        words = [snow.stem(word) for word in sentence.split(' ') if word not in stopwords.words('english')]   # Stemming and removing stopwords
        temp.append(sentence)

    count_vect = CountVectorizer(max_features=5000)
    bow_data = count_vect.fit_transform(temp)

    final_tf = temp
    tf_idf = TfidfVectorizer(ngram_range=(1,2),stop_words = 'english',max_features=5000)
    tf_data = tf_idf.fit_transform(final_tf)
    w2v_data = temp
    splitted = []
    for row in w2v_data: 
        splitted.append([word for word in row.split()])     #splitting words
    
    train_w2v = Word2Vec(splitted,min_count=5,size=50, workers=4)
    avg_data = []
    for row in splitted:
        vec = np.zeros(50, dtype=float)
        count = 0
        for word in row:
            try:
                vec += train_w2v[word]
                count += 1
            except:
                pass
        if (count == 0):
            avg_data.append(vec)
        else:
            avg_data.append(vec/count)
#        avg_data.append(vec)

    tf_w_data = []
    tf_data = tf_data.toarray()
    i = 0
    for row in splitted:
        vec = [0 for i in range(50)]
    
        temp_tfidf = []
        for val in tf_data[i]:
            if val != 0:
                temp_tfidf.append(val)
    
        count = 0
        tf_idf_sum = 0
        for word in row:
            try:
                count += 1
                tf_idf_sum = tf_idf_sum + temp_tfidf[count-1]
                vec += (temp_tfidf[count-1] * train_w2v[word])
            except:
                pass
        if (tf_idf_sum == 0):
            tf_w_data.append(vec)
        else:
            tf_w_data.append(vec/tf_idf_sum)
#            vec = float(1/tf_idf_sum) * vec
#        tf_w_data.append(vec)
        i = i + 1
    
    return tf_w_data

In [ ]:
#test cell
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

temp['descrption'] = test_embedding(description_X)
temp.descrption.head()
#tfidf_n = TfidfVectorizer(ngram_range=(1,2),stop_words = 'english')

In [5]:
def clean_data(filename):
    #clean up data for lvl 1&2&3
    filename['lvl1'] = filename['lvl1'].str.lower().replace('[^a-zA-Z]+',' ',regex=True)
    filename['lvl2'] = filename['lvl2'].str.lower().replace('[^\'\w]+',' ',regex=True)
    filename['lvl3'] = filename['lvl3'].str.lower().replace('[^\'\w]+',' ',regex=True)
    filename['descrption'] = filename['descrption'].str.lower()
    filename['name'] = filename['name'].str.lower()
    
    mapping_lvl1 = map_mathod('lvl1')
    mapping_lvl2 = map_mathod('lvl2')
    mapping_lvl3 = map_mathod('lvl3')
    
    filename['lvl1'] = filename['lvl1'].map(mapping_lvl1)
    filename['lvl2'] = filename['lvl2'].map(mapping_lvl2)
    filename['lvl3'] = filename['lvl3'].map(mapping_lvl3)
    
    #normalize price
    maxp = filename.price.max()
    valuethred = 1000.
    filename['price'] = filename['price'].clip(lower=0.,upper=valuethred)
    #filename['price'] = filename['price'].clip(lower=0.,upper=valuethred).div(valuethred,fill_value=None)
    #hist = train_file['price'].hist(bins=10)
    #maxp

    #clean up type 
    mapping_type = {'international':1.,'local':2., np.nan:0.}
    filename['type'] = filename['type'].map(mapping_type)
    
    #clean up text
    description_X = filename.descrption.str.lower().replace('<.*?>','',regex=True).replace('[^\w\s]+',' ',regex=True)
    filename['descrption'] = text_embedding(description_X)
    
    name_X = filename.name.str.lower().replace('<.*?>','',regex=True).replace('[^\w\s]+',' ',regex=True)
    filename['name'] = text_embedding(name_X)

    return filename,mapping_lvl1,mapping_lvl2,mapping_lvl3

In [6]:
train_file = load_data('train_data.csv')
cleaned_train,mapping_lvl1,mapping_lvl2,mapping_lvl3 = clean_data(train_file)
train_score = load_label('train_label.csv')

In [ ]:
# test cell
cleaned_train['descrption'] = temp['descrption']

In [7]:
cleaned_train.head(10)
#print(mapping_lvl1)


,name,lvl1,lvl2,lvl3,descrption,price,type
id,,,,,,,
1,"[-0.07032404059363115, -0.29758781334385276, -...",1.0,1.0,1.0,"[0.06992685375735164, -0.16184532611320415, -0...",128.00,1.0
2,"[0.8263886688011033, -0.3248656073618414, -1.3...",2.0,2.0,2.0,"[0.016663137124851346, -0.06203396766795777, 0...",14.69,1.0
3,"[1.4067267007406123, 0.039988674101716075, -2....",3.0,3.0,3.0,"[-0.09603992232587188, 0.3617549417540431, 0.4...",14.10,1.0
4,"[1.6705077759483282, 0.06829343867652557, -2.7...",3.0,3.0,3.0,"[0.10783753912065826, -0.06589401812981004, 0....",17.94,1.0
5,"[1.0321451897376819, 0.016257136293193874, -1....",3.0,3.0,3.0,"[0.3669052943587303, -0.943045073085361, -0.82...",6.80,1.0
6,"[-0.08606704908351485, -0.44580006927404053, -...",4.0,4.0,4.0,"[-0.3062274492560671, -0.22138636959537578, -0...",388.99,1.0
7,"[0.6027595897150391, -0.12192046743653276, -1....",2.0,5.0,5.0,"[0.1515675337545367, 0.02379527411216663, 0.11...",1000.00,2.0
8,"[0.09512590244412422, -0.850517663690779, -1.7...",5.0,6.0,6.0,"[-0.4776106401631195, -0.12558511003024048, -0...",21.74,1.0
9,"[0.14016459241393023, -0.13052392018726094, -1...",6.0,7.0,7.0,"[0.16502526154131084, -0.2545379170147266, -0....",25.00,2.0


In [8]:
from sklearn import preprocessing

def rearrange(cleaned_data):
    la = cleaned_data.lvl1.as_matrix(columns=None).tolist()
    lb = cleaned_data.lvl2.as_matrix(columns=None).tolist()
    lc = cleaned_data.lvl3.as_matrix(columns=None).tolist()

    X = la
    X = np.column_stack((X,lb))
    X = np.column_stack((X,lc))

    enc = preprocessing.OneHotEncoder()
    enc.fit(X)
    X = enc.transform(X).toarray()
    
    ld = cleaned_data.price.as_matrix(columns=None).tolist()
    le = cleaned_data.type.as_matrix(columns=None).tolist()

    X = np.column_stack((X,ld))
    X = np.column_stack((X,le))

   
    lf = cleaned_data.name.as_matrix(columns=None).tolist()
    lg = cleaned_data.descrption.as_matrix(columns=None).tolist()
    lg_array = np.vstack( lg )
    lf_array = np.vstack( lf )

    U, s, Vh = LA.svd(lg_array, full_matrices=False)
    assert np.allclose(lg_array, np.dot(U, np.dot(np.diag(s), Vh)))

# only use U \cdot s
    
#    s[8:] = 0.
#    new_lg = np.dot(U, np.dot(np.diag(s), Vh))
    s_new = s[:5]
    U_new = U[:, :5]
    new_lg = np.dot(U_new, np.diag(s_new))
    lg_min = np.min(new_lg)
    lg_out = new_lg  - (lg_min-2)*np.ones_like(new_lg.size)

    Uf, sf, Vhf = LA.svd(lf_array, full_matrices=False)
    assert np.allclose(lf_array, np.dot(Uf, np.dot(np.diag(sf), Vhf)))

#    sf[5:] = 0.
#    new_lf = np.dot(Uf, np.dot(np.diag(sf), Vhf))
    sf_new = sf[:5]
    Uf_new = Uf[:, :5]
    new_lf = np.dot(Uf_new, np.diag(sf_new))
    lf_min = np.min(new_lf)
    lf_out = new_lf  - (lf_min-2)*np.ones_like(new_lf.size)

    
    X = np.column_stack((X,lf_out))
    X = np.column_stack((X,lg_out))
    X = X.tolist()
   
    return X,lf_min,lg_min

    
    #print(len(X))
X,lf_min,lg_min = rearrange(cleaned_train)
print(np.shape(np.array(X)))
Y = train_score.score.as_matrix(columns=None).tolist()

(18141, 263)


In [9]:
np.min(X)


0.0

In [10]:
from sklearn.utils import shuffle
X, Y = shuffle(X, Y)
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(X, Y, test_size=0.20, random_state=0)

from keras.preprocessing import sequence
maxlen = 263
X_train = sequence.pad_sequences(X_train, maxlen=maxlen, dtype='float32')
X_validation = sequence.pad_sequences(X_validation, maxlen=maxlen, dtype='float32')
#X_train = np.any(np.isnan(X_train))
#X_train = np.all(np.isfinite(X_train))
print(X_train[1400].size)

Using TensorFlow backend.


263


In [11]:
import os
import time
from sklearn import metrics
from sklearn import preprocessing
import numpy as np
import pandas as pd
import random
import math
import warnings
warnings.filterwarnings("ignore")

from matplotlib import pyplot as plt

from sklearn import preprocessing
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import chi2

from sklearn import linear_model
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.ensemble import BaggingClassifier

from sklearn.model_selection import cross_val_score

In [12]:
# Linear Regression Classifier
def linear_regression_classifier(train_x, train_y):
    model = linear_model.LinearRegression()
    model.fit(train_x, train_y)
    return model
 
# Multinomial Naive Bayes Classifier
def naive_bayes_classifier(train_x, train_y):
    model = MultinomialNB()

    param_grid = {'alpha': [math.pow(10,-i) for i in range(11)]}
    grid_search = GridSearchCV(model, param_grid, n_jobs = 1, verbose=1)
    grid_search.fit(train_x, train_y)
    best_parameters = grid_search.best_estimator_.get_params()
    
    model = MultinomialNB(alpha = best_parameters['alpha'])  
    model.fit(train_x, train_y)
    return model
 
 
# KNN Classifier
def knn_classifier(train_x, train_y):
    model = KNeighborsClassifier()

    param_grid = {'n_neighbors': list(range(1,21))}
    grid_search = GridSearchCV(model, param_grid, n_jobs = 1, verbose=1)
    grid_search.fit(train_x, train_y)
    best_parameters = grid_search.best_estimator_.get_params()
    
    model = KNeighborsClassifier(n_neighbors = best_parameters['n_neighbors'], algorithm='kd_tree')

    bagging = BaggingClassifier(model, max_samples=0.5, max_features=1 )
    bagging.fit(train_x, train_y)
    return bagging
 
 
# Logistic Regression Classifier
def logistic_regression_classifier(train_x, train_y):
    model = LogisticRegression(penalty='l2')
    model.fit(train_x, train_y)
    return model
 
 
# Random Forest Classifier
def random_forest_classifier(train_x, train_y):
    model = RandomForestClassifier()

    param_grid = {'n_estimators': list(range(1,21))}
    grid_search = GridSearchCV(model, param_grid, n_jobs = 1, verbose=1)
    grid_search.fit(train_x, train_y)
    best_parameters = grid_search.best_estimator_.get_params()
    
    model = RandomForestClassifier(n_estimators = best_parameters['n_estimators'])
    
    model.fit(train_x, train_y)
    return model
 
 
# Decision Tree Classifier
def decision_tree_classifier(train_x, train_y):
    model = tree.DecisionTreeClassifier()
    model.fit(train_x, train_y)

    bagging = BaggingClassifier(model, max_samples=0.5, max_features=1 )
    bagging.fit(train_x, train_y)
    return bagging
 
 
# GBDT(Gradient Boosting Decision Tree) Classifier
def gradient_boosting_classifier(train_x, train_y):
    model = GradientBoostingClassifier()
    
    model = RandomForestClassifier()

    param_grid = {'n_estimators': list(range(100,300,10))}
    grid_search = GridSearchCV(model, param_grid, n_jobs = 1, verbose=1)
    grid_search.fit(train_x, train_y)
    best_parameters = grid_search.best_estimator_.get_params()
    
    model = RandomForestClassifier(n_estimators = best_parameters['n_estimators'])

    model.fit(train_x, train_y)
    return model

# SVM Classifier
def svm_classifier(train_x, train_y):
    model = SVC(kernel='linear', probability=True)
    model.fit(train_x, train_y)
    return model
 
# SVM Classifier using cross validation
def svm_cross_validation(train_x, train_y):
    model = SVC(kernel='linear', probability=True)
    param_grid = {'C': [1e-3, 1e-2, 1e-1, 1, 10, 100, 1000], 'gamma': [0.001, 0.0001]}
    grid_search = GridSearchCV(model, param_grid, n_jobs = 1, verbose=1)
    grid_search.fit(train_x, train_y)
    best_parameters = grid_search.best_estimator_.get_params()
    #for para, val in best_parameters.items():
        #print para, val
    model = SVC(kernel='rbf', C=best_parameters['C'], gamma=best_parameters['gamma'], probability=True)
    model.fit(train_x, train_y)
    return model

def feature_select(x,y):
    clf = ExtraTreesClassifier()
    clf = clf.fit(x, y)
    model = SelectFromModel(clf, prefit=True)
    return model

In [ ]:
# just for my own record


if __name__ == '__main__': 
    thresh = 0.5    
#    model_save_file = "/home/jason/datamining/model/models"     
#    model_save = {}
#    result_save_file = '/home/jason/datamining/result/results' 
     
    test_classifiers = ['KNN','LR','RF','DT']    
    classifiers = {
                   'KNN':knn_classifier,
                    'LR':logistic_regression_classifier,
                    'RF':random_forest_classifier,
                    'DT':decision_tree_classifier,                 
    }
        
    print('reading training and testing data...')    
    #X_train, X_validation, y_train, y_validation
    select_model = feature_select(X_train, y_train)
    X_train = select_model.transform(X_train)
    X_validation = select_model.transform(X_validation)

    result = []
        
    for classifier in test_classifiers:    
        print('******************* %s ********************' % classifier)    
        start_time = time.time()    
        model = classifiers[classifier](X_train, y_train)   
        print('training took %fs!' % (time.time() - start_time))    
        predict = model.predict(X_validation)

        precision = metrics.precision_score(y_validation, predict)    
        recall = metrics.recall_score(y_validation, predict)    
        print('precision: %.2f%%, recall: %.2f%%' % (100 * precision, 100 * recall))    
        accuracy = metrics.accuracy_score(y_validation, predict)    
        print('accuracy: %.2f%%' % (100 * accuracy))

        scores = cross_val_score(model, X_train, y_train)
        print(scores)

In [13]:
X_train
#X_train, y_train = X,Y

array([[ 0.       ,  0.       ,  0.       , ...,  6.121397 ,  8.0434885,
         8.24141  ],
       [ 0.       ,  0.       ,  0.       , ...,  8.813916 ,  7.5686784,
         9.206636 ],
       [ 0.       ,  0.       ,  1.       , ...,  8.532886 ,  8.417407 ,
         7.542454 ],
       ...,
       [ 1.       ,  0.       ,  0.       , ...,  8.556288 ,  7.9783664,
         9.911194 ],
       [ 0.       ,  0.       ,  1.       , ...,  7.0517106,  9.395992 ,
         8.676114 ],
       [ 0.       ,  0.       ,  1.       , ..., 10.234994 , 12.946437 ,
         8.970349 ]], dtype=float32)

In [14]:
#test cnn model

from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D
from keras.datasets import imdb

# Embedding
max_features = 15000
maxlen = 263
embedding_size = 128

# Convolution
kernel_size = 5
filters = 64
pool_size = 3

# LSTM
lstm_output_size = 70

# Training
batch_size = 30
epochs = 5

'''
Note:
batch_size is highly sensitive.
Only 2 epochs are needed as the dataset is very small.
'''

print('Loading data...')
#X_train, X_validation, y_train, y_validation
#(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

#X_train = np.asarray(np.abs(X))
X_train = np.asarray(np.abs(X_train))
X_validation = np.asarray(np.abs(X_validation))
print(len(X_train), 'train sequences')
print(len(X_validation), 'test sequences')

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=maxlen, padding='post')
X_validation = sequence.pad_sequences(X_validation, maxlen=maxlen, padding='post')
print('x_train shape:', X_train.shape)
print('x_test shape:', X_validation.shape)

print('Build model...')

cnnmodel = Sequential()
cnnmodel.add(Embedding(max_features, embedding_size,input_length=maxlen))
#model.add(Dense(32, activation='relu', input_dim=100))
cnnmodel.add(Dropout(0.5))
cnnmodel.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
cnnmodel.add(MaxPooling1D(pool_size=pool_size))
cnnmodel.add(LSTM(lstm_output_size))
cnnmodel.add(Dense(1))
cnnmodel.add(Activation('sigmoid'))

cnnmodel.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


Loading data...
14512 train sequences
3629 test sequences
Pad sequences (samples x time)
x_train shape: (14512, 263)
x_test shape: (3629, 263)
Build model...


In [15]:
test_file = load_data('test_data.csv')
#cleaned_test = clean_data(test_file)
X_train, y_train = X,Y

In [16]:
filename = test_file
#clean up data for lvl 1&2&3
filename['lvl1'] = filename['lvl1'].str.lower().replace('[^a-zA-Z]+',' ',regex=True)
filename['lvl2'] = filename['lvl2'].str.lower().replace('[^\'\w]+',' ',regex=True)
filename['lvl3'] = filename['lvl3'].str.lower().replace('[^\'\w]+',' ',regex=True)
filename['descrption'] = filename['descrption'].str.lower()
filename['name'] = filename['name'].str.lower()
    
#    mapping_lvl1 = map_mathod('lvl1')
#    mapping_lvl2 = map_mathod('lvl2')
#    mapping_lvl3 = map_mathod('lvl3')
    
filename['lvl1'] = filename['lvl1'].map(mapping_lvl1)
filename['lvl2'] = filename['lvl2'].map(mapping_lvl2)
filename['lvl3'] = filename['lvl3'].map(mapping_lvl3)
    
    #normalize price
maxp = filename.price.max()
valuethred = 1000.
filename['price'] = filename['price'].clip(lower=0.,upper=valuethred)
#filename['price'] = filename['price'].clip(lower=0.,upper=valuethred).div(valuethred,fill_value=None)
    #hist = train_file['price'].hist(bins=10)
    #maxp

    #clean up type 
mapping_type = {'international':1.,'local':2., np.nan:0.}
filename['type'] = filename['type'].map(mapping_type)
    
    #clean up text
description_X = filename.descrption.str.lower().replace('<.*?>','',regex=True).replace('[^\w\s]+',' ',regex=True)
filename['descrption'] = text_embedding(description_X)
    
name_X = filename.name.str.lower().replace('<.*?>','',regex=True).replace('[^\w\s]+',' ',regex=True)
filename['name'] = text_embedding(name_X)

#    return filename

In [17]:
cleaned_test = filename
lat = cleaned_test.lvl1.as_matrix(columns=None).tolist()
lbt = cleaned_test.lvl2.as_matrix(columns=None).tolist()
lct = cleaned_test.lvl3.as_matrix(columns=None).tolist()

Xt = lat
Xt = np.column_stack((Xt,lbt))
Xt = np.column_stack((Xt,lct))

enct = preprocessing.OneHotEncoder()
enct.fit(Xt)
Xt = enct.transform(Xt).toarray()

ldt = cleaned_test.price.as_matrix(columns=None).tolist()
let = cleaned_test.type.as_matrix(columns=None).tolist()

Xt = np.column_stack((Xt,ldt))
Xt = np.column_stack((Xt,let))

lft = cleaned_test.name.as_matrix(columns=None).tolist()
lgt = cleaned_test.descrption.as_matrix(columns=None).tolist()
lg_arrayt = np.vstack( lgt )
lf_arrayt = np.vstack( lft )

Ut, st, Vht = LA.svd(lg_arrayt, full_matrices=False)
assert np.allclose(lg_arrayt, np.dot(Ut, np.dot(np.diag(st), Vht)))

st_new = st[:5] 
Ut_new = Ut[:, :5]
new_lgt = np.dot(Ut_new, np.diag(st_new))
lgt_out = new_lgt  - (lg_min-2)*np.ones_like(new_lgt.size)

Uf, sf, Vhf = LA.svd(lf_arrayt, full_matrices=False)
assert np.allclose(lf_arrayt, np.dot(Uf, np.dot(np.diag(sf), Vhf)))

sf_new = sf[:5] 
Uf_new = Uf[:, :5]
new_lft = np.dot(Uf_new, np.diag(sf_new))
lft_out = new_lft  - (lf_min-2)*np.ones_like(new_lft.size)


#X = la
#X = np.column_stack((X,lb))
#X = np.column_stack((X,lc))
#X = np.column_stack((X,ld))
#X = np.column_stack((X,le))
Xt = np.column_stack((Xt,lft_out))
Xt = np.column_stack((Xt,lgt_out))
Xt = Xt.tolist()
#cleaned_test = Xt
#return X,s

In [18]:
#cleaned_test = clean_test(test_file)
#cleaned_test = filename
cleaned_test.head(10)
np.min(Xt)

0.0

In [19]:
#from keras.preprocessing import sequence
#maxlen = 150
#X_test = rearrange(cleaned_test)
X_test = np.asarray(Xt)
#X_train = sequence.pad_sequences(X_train, maxlen=maxlen, dtype='float32')
#X_test = sequence.pad_sequences(X_test, maxlen=maxlen, dtype='float32')

In [20]:
if __name__ == '__main__': 
    thresh = 0.5    
#    model_save_file = "/home/jason/datamining/model/models"     
#    model_save = {}
#    result_save_file = '/home/jason/datamining/result/results' 
     
    test_classifiers = ['LR','RF']    
    classifiers = {
                   'LR':logistic_regression_classifier,
                   'RF':random_forest_classifier         
    }
        
    print('reading training and testing data...')    
    #X_train, X_validation, y_train, y_validation
#    X_test = rearrange(Xt)
#    X_test = Xt
    select_model = feature_select(X_train, y_train)
    X_train = select_model.transform(X_train)
    X_test = select_model.transform(X_test)

    result = []
    
    start_time = time.time()
    model = classifiers['LR'](X_train, y_train)   
    print('training took %fs!' % (time.time() - start_time))    
    Y_predict_lr = model.predict_proba(X_test)[:,1]
    print('predict finished')
    
    model = classifiers['RF'](X_train, y_train)   
    print('training took %fs!' % (time.time() - start_time))    
    Y_predict_rf = model.predict_proba(X_test)[:,1]
    print('predict finished')
    
        
#    for classifier in test_classifiers:    
#        print('******************* %s ********************' % classifier)    
#        start_time = time.time()    
#        model = classifiers[classifier](X_train, y_train)   
#        print('training took %fs!' % (time.time() - start_time))    
#        Y_predict = model.predict_proba(X_test)[:,1]
#        print('predict finished')

reading training and testing data...
training took 0.346484s!
predict finished
Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   17.9s finished


training took 19.723191s!
predict finished


In [21]:
#cnn test
X_train, y_train = X,Y
X_train = np.asarray(X_train)
X_train = sequence.pad_sequences(X_train, maxlen=maxlen, padding='post')
#X_test = np.asarray(Xt)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen, padding='post')


cnnmodel.fit(X_train, y_train, batch_size=128, epochs=5)

Y_predict_cnn = cnnmodel.predict(X_test, verbose=0)
Y_predict_cnn = np.squeeze(Y_predict_cnn, axis=1)

Epoch 1/5
18141/18141 [==============================] - 55s 3ms/step - loss: 0.5784 - acc: 0.7033
Epoch 2/5
18141/18141 [==============================] - 54s 3ms/step - loss: 0.5353 - acc: 0.7355
Epoch 3/5
18141/18141 [==============================] - 54s 3ms/step - loss: 0.5236 - acc: 0.7430
Epoch 4/5
18141/18141 [==============================] - 54s 3ms/step - loss: 0.5132 - acc: 0.7480
Epoch 5/5
18141/18141 [==============================] - 55s 3ms/step - loss: 0.5044 - acc: 0.7502


In [22]:
Y_predict_cnn

array([0.5098013 , 0.5097091 , 0.50959545, ..., 0.5096364 , 0.5095682 ,
       0.5094789 ], dtype=float32)

In [23]:
Y_predict_lr

array([0.70960149, 0.70658239, 0.72379223, ..., 0.70881897, 0.43334579,
       0.38980332])

In [24]:
Y_predict_rf

array([0.70588235, 0.58823529, 0.47058824, ..., 0.70588235, 0.41176471,
       0.23529412])

In [25]:
Y_predict = []
for i in range(len(Y_predict_rf)):
    Y_predict.append((Y_predict_lr[i] + Y_predict_rf[i] + Y_predict_cnn[i]) / 3.0)

In [26]:
Y_predict

[0.6417617229753266,
 0.601508934407967,
 0.5679919742959664,
 0.7383062791580081,
 0.6171568150835252,
 0.6697510249074924,
 0.6363602936944943,
 0.5993328340575735,
 0.6324364086201705,
 0.6024202375325002,
 0.7412061216070346,
 0.5039924644737895,
 0.6911224905837831,
 0.5497889798337453,
 0.7285973515173213,
 0.4322579983611576,
 0.7597256810459195,
 0.5693371582091432,
 0.6584914961098802,
 0.5813136034607614,
 0.5060994748528079,
 0.6728534855827433,
 0.7699855760930033,
 0.6794054164001916,
 0.7245350858787151,
 0.6663921197853707,
 0.6474549129019334,
 0.7260616324030026,
 0.7449465674147385,
 0.7080520196835809,
 0.6602019190695558,
 0.6189760945354279,
 0.8018967271664402,
 0.6533012547860929,
 0.6731650656933224,
 0.44273150031352965,
 0.6556826477302232,
 0.6946488275468113,
 0.49863724811481697,
 0.7189814859771816,
 0.49013529480044477,
 0.7680072962577363,
 0.5581324252460774,
 0.48171598721118625,
 0.6879753460645732,
 0.6069928273612174,
 0.6541088264043867,
 0.3565465

In [27]:
Y_predict_temp = []
for i in range(len(Y_predict_rf)):
    Y_predict_temp.append((Y_predict_lr[i] + Y_predict_rf[i]) / 2.0)

In [28]:
Y_predict_temp

[0.7077419203718798,
 0.6474088417135742,
 0.5971902345743665,
 0.8525318431487858,
 0.6709162466998422,
 0.749755824604128,
 0.6996963114562007,
 0.6441616785117448,
 0.69376536312862,
 0.648728768335646,
 0.8568564238599079,
 0.501205334560492,
 0.7817839956565584,
 0.5698564770004502,
 0.8380179831475639,
 0.39355046804840443,
 0.8846866057802287,
 0.5991760921568544,
 0.7329030690549571,
 0.6172307630478926,
 0.5042821950100773,
 0.754448616263977,
 0.9001043433246093,
 0.7642034371956548,
 0.831840601745166,
 0.7446599782410597,
 0.7163125208631387,
 0.8341837080840266,
 0.8625470522024302,
 0.8071699446559866,
 0.7355361759523562,
 0.6736070487073624,
 0.9479326249388846,
 0.7251243529871685,
 0.7549185792318933,
 0.4093566546527285,
 0.7286298044105051,
 0.7871072075277212,
 0.49309657370458937,
 0.8235617003059519,
 0.48044732601261786,
 0.8971174030505449,
 0.582346104528662,
 0.46780528140920186,
 0.777134953463086,
 0.655595729508089,
 0.7262703241192907,
 0.2800641920745904

In [30]:
temp_score = load_label('submission.csv')
submit_score  = temp_score
submit_score['score'] = Y_predict
submit_score.to_csv('predict_result_cnn.csv')

In [31]:
temp_score = load_label('submission.csv')
submit_score  = temp_score
submit_score['score'] = Y_predict_temp
submit_score.to_csv('predict_result_ncnn.csv')

In [ ]:
table = pd.pivot_table(train_file,values = 'price', index=['lvl1','lvl2','lvl3'],columns=['type'],aggfunc=[min, max, np.mean])
table